In [1]:
from string import punctuation as PUNCT

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
STOPWORDS = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

Реализация алгоритма.  
Я также добавил возможность удаления стоп-слов из определений дизамбигуируемого слова и стемминга как определений, так и слов контекста.  
Возвращается кортеж *(индекс синсета с лучшим определением, длина пересечения, содержание лучшего определения)*, если `return_def=True`, иначе возвращается только индекс синсета с лучшим определением.  
Ф-ия `apply_lesk` позволяет дизамбигуировать все слова во входной строке.

In [12]:
def lesk(target, context, remove_stopwords=True, stemming=True, return_def=True):
    """
    target: <str>
    context: List[str]
    remove_stopwords: bool: remove stopwords in definitions
    stemming: bool: stem words in definitions
    
    return: idx of the best synset if `return_def=False` else (idx, overlap, definition)
    """

    target_synsets = [_synset for _synset in wn.synsets(target)]
    target_defs = [set(_synset.definition().lower().split()) for _synset in target_synsets]
    context = [w.lower() for w in context]
    
    if remove_stopwords:
        target_defs = [x.difference(STOPWORDS) for x in target_defs]
    
    if stemming:
        target_defs = [set([stemmer.stem(_x) for _x in x]) for x in target_defs]
        context = [stemmer.stem(w) for w in context]
    
    sense_scores = {i: (len(_def.intersection(context)), target_synsets[i])
                    for i, _def in enumerate(target_defs)
    }
    
    if not sense_scores:
        sense_scores = {-1: (0, None)}
    
    best_sense = sorted(sense_scores, key=lambda x: sense_scores.get(x)[0], reverse=True)[0]
    
    return (best_sense, *sense_scores[best_sense]) if return_def else best_sense


def get_context(words, window=3):
    for i, word in enumerate(words):
        yield (word, words[max(0, i-window):i] + words[i+1:i+1+window])


def apply_lesk(text, window=3, remove_stopwords=True, lesk_params={}):
    """
    return: (word, best_sense_idx, overlap, definition)
    """
    
    text = [w.lower() for w in word_tokenize(text) if w.strip(PUNCT)]
    text = [w for w in text if w not in STOPWORDS] if remove_stopwords else text
    
    return [(word, *lesk(word, context, **lesk_params)) for word, context in get_context(text, window)]
 

Проверим алгоритм на примере

In [13]:
sent = 'I went to the bank to deposit money.'

In [17]:
lesk('bank', ['went', 'money', 'deposit'])

(1, 2, Synset('depository_financial_institution.n.01'))

Проверим на предложении

In [15]:
apply_lesk(sent, remove_stopwords=True)

[('went', 0, 0, Synset('travel.v.01')),
 ('bank', 1, 2, Synset('depository_financial_institution.n.01')),
 ('deposit', 3, 2, Synset('deposit.n.04')),
 ('money', 2, 1, Synset('money.n.03'))]

Весь ввод на вывод

In [16]:
apply_lesk(sent, remove_stopwords=False)

[('i', 0, 0, Synset('iodine.n.01')),
 ('went', 0, 0, Synset('travel.v.01')),
 ('to', -1, 0, None),
 ('the', -1, 0, None),
 ('bank', 1, 2, Synset('depository_financial_institution.n.01')),
 ('to', -1, 0, None),
 ('deposit', 3, 2, Synset('deposit.n.04')),
 ('money', 2, 1, Synset('money.n.03'))]

Выдача вида `(-1, 0, None)` означает, что в WordNet нет синсетов для данного слова